# Section of region of interest

In this notebook, we create the regions of interest where nuclei will be segregated into later.

We start to load the file at a low resolution and create regions layers in napari.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import octofish
import yaml
from pathlib import Path
from ipyfilechooser import FileChooser
srcdir, dstdir = '', ''
if Path("config.yml").exists():
    with open("config.yml", "r") as file:    
        config = yaml.safe_load(file)
        if 'source' in config.keys():
            srcdir = Path(config["source"])        
        if 'destination' in config.keys():
            dstdir = Path(config["destination"]) 

fc = FileChooser(dstdir, select_desc='destination')
display(fc)



Check the list of files defined at step 1

In [ ]:
import pandas as pd
dstdir = Path(fc.selected) if fc.selected is not None else Path(dstdir)
filelistname = dstdir / 'filelist.csv'
filelist = pd.read_csv(filelistname)
filelist

Select a file from the list if necessary.

In [ ]:
import ipywidgets as widgets
if len(filelist) == 1:
    print("Filelist has only one item.")
    w = None
else:
    w = widgets.Dropdown(
        options=[(x,k) for k,x in enumerate(filelist['name'])],
        value=1,
        description='Image:',
    )
    display(w)

Load the selected file and update config.yml

In [ ]:
row = filelist.iloc[w.value] if w is not None  else filelist.iloc[0]
filename = octofish.get_files(dstdir, row, 'ims')
regionname = octofish.get_files(dstdir, row, 'regions')
print("Input IMS filename:", filename)
print("Destination:", dstdir)
print("Output region file:", regionname)
# update yaml config file
try:
    cnf = yaml.safe_load(f'source: \"{srcdir.as_posix()}\"\ndestination: \"{dstdir.as_posix()}\"\n')
    with open('config.yml', 'w') as file:
        yaml.dump(cnf, file)
except:
    pass


Load the file from disk, change the resolution level if needed

In [ ]:
resolution_level = 1
nuclei = octofish.load_nuclei_channel_2d(dstdir, row, resolution_level)

Load the file in napari and add manually some regions. 

If there are existing regions, load the regions as shapes.

In [ ]:

import napari

v = napari.view_image(nuclei)

rois = octofish.load_region_polygons(dstdir, row, resolution_level)
if rois is not None:
    v.add_shapes(rois, shape_type='polygon')


Export the region to a json file

In [ ]:
octofish.save_regions(dstdir, row, resolution_level, v.layers[-1].data )

Reload the regions to check for correctness

In [ ]:
import matplotlib.pyplot as plt
rois = octofish.load_regions_as_labels(dstdir, row, nuclei.shape, resolution_level)
plt.imshow(nuclei)
plt.imshow(rois, alpha=0.5)
plt.axis('off');